## Lista 3 - Aprendizagem de máquina probabilístico
- Aluno: Lucas Rodrigues Aragão - Graduação 538390

## Solução MAP para modelos de mistura gaussiana usando Expectation Maximization

1. Escolher os hiperparâmetros $K, \alpha_k, \mu_0, k_0, \pi^{(0)}_k, \mu_k^{(0)}, \Sigma^{0}_{k}$

Valores comuns: $S_0 = \frac{1}{K^{2/D}} S_{\bar{x}}$, $v_0 = D+2$, $\mu_0 = \bar{x}$ e $k_0 = 0.01$. Em que $\bar{x}$ é a média amostral e $S_{\bar{x}}$ é a covariância amostral.

### Expectation

$$r_{ik} \triangleq p(z_i = k| x_i, \theta^{(t-1)}) = \frac{\pi_k^{(t-1)} \mathcal{N}(x_i| \mu_k^{(t-1)}, \Sigma_{k}^{(t-1)})}{\sum^{K}_{k'} \pi^{(t-1)}_{k'} \mathcal{N}(x_i| \mu_{k'}^{(t-1)}, \Sigma_{k'}^{(t-1)})} $$

### Maximization

$$\pi_k^{(t)} = \frac{\alpha_k - 1 + \Sigma_i r_{ik}}{N-K + \Sigma_k \alpha_k}$$

$$\bar{x_k} \triangleq \frac{\Sigma_i r_{ik}x_i}{\Sigma_i r_{ik}}$$

$$\mu_k^{(t)} = \frac{k_0m_0 + \bar{x}_k \Sigma_i r_{ik}}{k_0 + \Sigma_i r_{ik}}$$

$$\Sigma_k^{(t)} = \frac{S_0 + \Sigma_i r_{ik}(x_i - \bar{x}_k)^T + \frac{k_0 \Sigma_i r_{ik}}{k_0 + \Sigma_i r_{ik}} (\bar{x}_k - m_0) (\bar{x}_k -m_0)^T}{v_0 + D + 2 + \Sigma_i r_{ik}}$$


In [2]:
class EmGMM():
    def __init__(self, K, alphak, mu0, k0, pik,  ):
        self
        pass
    
    def expectation():
        pass

    def maximization():
        pass

    def MAP():
        pass
    

## Métricas de avaliação

Sejam M o número de parâmetros do modelo e N o total de obsevações.

### Bayesian Information Criterion

$$BIC(D,K) \triangleq \log p(D|\hat{\theta}) - \frac{1}{2} M \log N $$

### Akaike Information Criterion

$$AIC(D,K) \triangleq \log p(D|\hat{\theta}) - M$$

In [ ]:
def BIC():
    pass

def AIC():
    pass